In [ ]:
import pandas as pd

df = pd.read_csv('/workspaces/codespaces-jupyter/data/FTE Enrollment by Grade Fiscal Year2019-3 Data Report.csv', skiprows=5, encoding='latin1')
print(df.head())

In [ ]:
import plotly.express as px
# Identify grade columns
grade_columns = [col for col in df.columns if 'Grade' in col]

# Aggregate enrollment numbers across all schools
grade_totals = df[grade_columns].sum().reset_index()
grade_totals.columns = ['Grade', 'Enrollment']

# Grade ordering
grade_order = ['Grade PK', 'Grade KK'] + [f'Grade {str(i).zfill(2)}' for i in range(1, 13)]
grade_totals['Grade'] = pd.Categorical(grade_totals['Grade'], categories=grade_order, ordered=True)
grade_totals = grade_totals.sort_values('Grade')

# Interactive bar chart
fig = px.bar(
    grade_totals,
    x='Grade',
    y='Enrollment',
    title='District-wide Enrollment by Grade (FTE 2019-3)',
    labels={'Enrollment': 'Number of Students'},
    text_auto=True
)
fig.show()

In [ ]:
import plotly.express as px

# Clean column names
df.columns = df.columns.str.strip()

# Get top 20 schools by total enrollment
school_totals = df[['School Name', 'Total']].sort_values('Total', ascending=False).head(20)

# Create bar chart
fig = px.bar(
    school_totals,
    x='Total',
    y='School Name',
    orientation='h',
    title='Top 20 Schools by Total Enrollment',
    labels={'Total': 'Number of Students', 'School Name': 'School'}
)

fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()


#### Enrollment Distribution Across Grades

In [ ]:
df_long = df.melt(id_vars=['School Name'], value_vars=[col for col in df.columns if 'Grade' in col], 
                  var_name='Grade', value_name='Enrollment')
fig = px.box(df_long, x='Grade', y='Enrollment', title='Enrollment Distribution per Grade Across Schools')
fig.show()


### School Size Categorization

In [ ]:
bins = [0, 300, 600, 1000, 2000]
labels = ['Small (<300)', 'Medium (300–600)', 'Large (600–1000)', 'Very Large (>1000)']
df['Size Category'] = pd.cut(df['Total'], bins=bins, labels=labels)
fig = px.pie(df, names='Size Category', title='Distribution of School Sizes')
fig.show()


#### PK vs HS Enrollment Comparison

In [ ]:
df['Early Grades'] = df[['Grade PK', 'Grade KK']].sum(axis=1)
df['High School'] = df[['Grade 11', 'Grade 12']].sum(axis=1)

fig = px.scatter(df, x='Early Grades', y='High School',
                 hover_name='School Name', title='PK+KK vs. Grade 11–12 Enrollment by School')
fig.show()


#### Grade Transition Ratios

In [ ]:
df['Grade 04 to 05 Ratio'] = df['Grade 05'] / df['Grade 04']

#### Outlier Detection

In [ ]:
outliers = df[df['Grade 06'] > df['Grade 05'] * 2]